In [ ]:
import tensorflow as tf
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [ ]:
!pip install mtcnn

In [ ]:
!pip install lz4

In [ ]:
import cv2, os, pickle
import numpy as np
import tensorflow as tf
from mtcnn import MTCNN
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D
from tqdm import tqdm
from PIL import Image

In [ ]:
base_model = ResNet50(
    weights="imagenet",
    include_top=False,
    input_shape=(224,224,3)
)

base_model.trainable = False

In [ ]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
model = tf.keras.models.Sequential()
model.add(base_model)
# model.add(x)
model.add(tf.keras.layers.Dense(128, activation='relu'))

model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ resnet50 (Functional)           │ (None, 7, 7, 2048)     │    23,587,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 7, 7, 128)      │       262,272 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 23,849,984 (90.98 MB)

 Trainable params: 262,272 (1.00 MB)

 Non-trainable params: 23,587,712 (89.98 MB)

In [ ]:
detector = MTCNN()

def extract_face(img_path):
    img = cv2.imread(img_path)
    rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    faces = detector.detect_faces(rgb)

    if len(faces) == 0:
        return None

    x, y, w, h = faces[0]['box']
    face = rgb[y:y+h, x:x+w]
    face = cv2.resize(face, (224,224))
    face = face / 255.0
    return face

In [ ]:
embeddings = []
labels = []

dataset_path = "/content/drive/MyDrive/AIUB/CVPR_DATASET"

for student in os.listdir(dataset_path):
    student_path = os.path.join(dataset_path, student)

    for img_name in tqdm(os.listdir(student_path)):
        img_path = os.path.join(student_path, img_name)
        # face = extract_face(img_path)
        face = Image.open(img_path)
        face = face.resize((224, 224))
        face = np.array(face)
        face = face / 255.0

        if face is None:
            continue

        emb = model.predict(face[np.newaxis, ...], verbose=0)
        embeddings.append(emb[0])
        labels.append(student)

100%|██████████| 20/20 [00:01<00:00, 13.17it/s]


NotADirectoryError: [Errno 20] Not a directory: '/content/drive/MyDrive/AIUB/CVPR_DATASET/resnet_embedding_model.keras'

In [ ]:
model.save("resnet_embedding_model.keras")

with open("face_embeddings.pkl", "wb") as f:
    pickle.dump((embeddings, labels), f)

In [ ]:
print(len(embeddings))
print(len(embeddings[0]))

100
7


In [ ]:
len(labels)

100

In [ ]:
model.save("/content/drive/MyDrive/AIUB/CVPR_DATASET/resnet_embedding_model.keras")